In [2]:
import numpy as np
import os
import train_dense
import train_cnn

In [3]:
def load_mnist():
    data_dir = '../data'

    fd = open(os.path.join(data_dir, 'train-images-idx3-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    trX = loaded[16:].reshape((60000, 28, 28, 1)).astype(np.float)

    fd = open(os.path.join(data_dir, 'train-labels-idx1-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    trY = loaded[8:].reshape((60000)).astype(np.int)

    fd = open(os.path.join(data_dir, 't10k-images-idx3-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    teX = loaded[16:].reshape((10000, 28, 28, 1)).astype(np.float)

    fd = open(os.path.join(data_dir, 't10k-labels-idx1-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    teY = loaded[8:].reshape((10000)).astype(np.int)

    trY = np.asarray(trY)
    teY = np.asarray(teY)

    perm = np.random.permutation(trY.shape[0])
    trX = trX[perm]
    trY = trY[perm]

    perm = np.random.permutation(teY.shape[0])
    teX = teX[perm]
    teY = teY[perm]

    return trX, trY, teX, teY

In [4]:
def print_digit(digit_pixels, label='?'):
    for i in range(28):
        for j in range(28):
            if digit_pixels[i, j] > 128:
                print('# ', end = '')
            else:
                print('. ', end = '')
        print('')

    print('Label: ', label)

In [5]:
trainX, trainY, testX, testY = load_mnist()

In [6]:
print("Shapes: ", trainX.shape, trainY.shape, testX.shape, testY.shape)

print("\nDigit sample")
print_digit(trainX[1], trainY[1])

Shapes:  (60000, 28, 28, 1) (60000,) (10000, 28, 28, 1) (10000,)

Digit sample
. . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . # # # . . . . . . . . . . . . . 
. . . . . . . . . . . # # # # # # . . . . . . . . . . . 
. . . . . . . . . . # # # # # # # . . . . . . . . . . . 
. . . . . . . . . # # . . . . # # # . . . . . . . . . . 
. . . . . . . . # # # . . . . . # # . . . . . . . . . . 
. . . . . . . . # # # . . . . . # # # . . . . . . . . . 
. . . . . . . . # # # # # # # # # # # . . . . . . . . . 
. . . . . . . . . # # # # # # # # # # . . . . . . . . . 
. . . . . . . . . . . . . . . # # # # . . . . . . . . . 
. . . . . . . . . . . . . . . # # # # . . . . . . . . . 
. . . . .

In [7]:
train_dense.train(trainX, trainY)
labels = train_dense.test(testX)
accuracy = np.mean((labels == testY)) * 100.0
print("\nDNN Test accuracy: %lf%%" % accuracy)


DNN Test accuracy: 9.800000%


#### Train and Test functions for dense networks

In [8]:
import tensorflow as tf

In [9]:
def test(testX):

    return np.zeros(testX.shape[0])

In [28]:
# def train(trainX, trainY):
def modify(X, Y):
    X_mod = np.ndarray((X.shape[0], 28*28))
    for i in range(X.shape[0]):
        X_mod[i, :, None] = np.reshape(X[i, :, :], (28*28, 1))
    
    Y_mod = np.zeros((X.shape[0], 10))
    for i in range(Y.shape[0]):
        Y_mod[i, Y[i]] = 1
    print(X_mod.shape)
    print(Y_mod.shape)
    
    return X_mod, Y_mod
    
trainX_mod, trainY_mod = modify(trainX, trainY)
testX_mod, testY_mod = modify(testX, testY)
    
# RUN
# train(trainX, trainY)

(60000, 784)
(60000, 10)
(10000, 784)
(10000, 10)


In [ ]:
learning_rate = 0.001
training_epochs = 5
batch_size = 100

input_dim = 28*28

n_input = input_dim
n_hidden_1 = int(input_dim/2) + 5
n_output = 10

x = tf.placeholder('float', [None, n_input])
y = tf.placeholder('float', [None, n_output])

def mperceptron(x, weights, biases):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)

    # layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # layer_2 = tf.nn.relu(layer_2)

    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    return out_layer

weights = {
    'h1' : tf.Variable(tf.random_normal([n_input, n_hidden_1], 0, 0.001)),
    # 'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], 0, 0.001)),
    'out' : tf.Variable(tf.random_normal([n_hidden_1, n_output], 0, 0.001))
}

biases = {
    'b1' : tf.Variable(tf.random_normal([n_hidden_1], 0, 0.001)),
    # 'b2': tf.Variable(tf.random_normal([n_hidden_2], 0, 0.001)),
    'out' : tf.Variable(tf.random_normal([n_output], 0, 0.001))
}

pred = mperceptron(x, weights, biases)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

init = tf.global_variables_initializer()

sess = tf.Session()    
sess.run(init)

In [32]:
epochs = 5
batches = 100
batch_size = int((trainX_mod.shape[0])/batches)

for _ in range(epochs):
    for bx in range(batches):
        batch_indices = list(range((bx*batch_size), (bx + 1)*batch_size))
        _, c = sess.run([optimizer, cost], \
                        feed_dict={x: trainX_mod[batch_indices, :],\
                                   y: trainY_mod[batch_indices]})

In [36]:
y_hat = sess.run(pred, feed_dict={x: testX_mod})

# Check performance
correct_prediction = sess.run(tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1)), feed_dict={x: testX_mod, y:testY_mod})
print(np.sum(correct_prediction), y_hat.shape[0])

9733 10000


In [21]:
# sess.close()

In [22]:
trainX_mod.shape

(60000, 784)

In [30]:
trainY_mod

array([[ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]])